In [ ]:
!pip install -U langchain-openai langchain

In [3]:
#define constante 
OPENAI_API_KEY= "YOUR_API_KEY"

# Prompt

installs

Clé API 

### Structure d'un Prompt

Une prompt peut comporter plusieurs éléments :

* Instructions
* Informations externes ou contexte
* Entrée ou requête de l'utilisateur
* Indicateur de sortie

Tous les prompt n'ont pas besoin de tous ces éléments, mais un bon prompt en utilise souvent au moins deux. Définissons plus précisément ce qu'ils sont:

Les **instructions** indiquent au modèle ce qu'il doit faire, généralement comment il doit utiliser les entrées et/ou les informations externes pour produire la sortie souhaitée.

**Les informations externes ou le contexte** sont des informations supplémentaires que nous insérons manuellement dans l'invite, que nous récupérons via une base de données vectorielle (mémoire à long terme) ou que nous obtenons par d'autres moyens (appels d'API, calculs, etc.).

**L'entrée ou la requête de l'utilisateur** est généralement une requête directement introduite par l'utilisateur du système.

**L'indicateur de sortie** est le début du texte généré. Pour un modèle générant du code Python, nous pouvons mettre `import ` (car la plupart des scripts Python commencent par un `import` de bibliothèque), ou un chatbot peut commencer par `Chatbot : ` (en supposant que nous formatons le script du chatbot comme des lignes de texte interchangées entre `User` et `Chatbot`).

Chacune de ces composantes doit généralement être placé dans l'ordre où nous les avons décrits. Nous commençons par les instructions, nous fournissons le contexte (si nécessaire), puis nous ajoutons l'entrée de l'utilisateur, et enfin nous terminons par l'indicateur de sortie.

exemple de prompt

In [3]:
prompt="""Réponds à la question grâce au contexte si dessous. S'il n'y a pas de réponses possible à la question avec les informations fournies, 
réponds simplement "jsp".

Context:Les grands modèles de langage (LLM) sont les modèles les plus récents utilisés dans le domaine du NLP.
Leurs performances supérieures à celles des modèles plus petits les ont rendus incroyablement utiles pour les développeurs d'applications NLP.
Ces modèles sont accessibles via la bibliothèque `transformers` de Hugging Face, via OpenAI en utilisant la bibliothèque `openai`, 
et via Cohere en utilisant la bibliothèque `cohere`.

Question: Quelle bibliothèque et fournisseurs de modèles proposent des LLMs?

Réponse:"""

In [5]:
from langchain_openai import OpenAI

# initialize the models
openai = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY
)
print(openai.invoke(prompt))

 "La bibliothèque `transformers` de Hugging Face, via OpenAI en utilisant la bibliothèque `openai`, et via Cohere en utilisant la bibliothèque `cohere`." 


## `PromptTemplate`

In [10]:
from langchain import PromptTemplate

template = """Réponds à la question grâce au contexte si dessous. S'il n'y a pas de réponses possible à la question avec les informations fournies, 
réponds simplement "jsp".

Context:Les grands modèles de langage (LLM) sont les modèles les plus récents utilisés dans le domaine du NLP.
Leurs performances supérieures à celles des modèles plus petits les ont rendus incroyablement utiles pour les développeurs d'applications NLP.
Ces modèles sont accessibles via la bibliothèque `transformers` de Hugging Face, via OpenAI en utilisant la bibliothèque `openai`, 
et via Cohere en utilisant la bibliothèque `cohere`.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [11]:
print(
    prompt_template.format(
        query="Quelle bibliothèque et fournisseurs de modèles proposent des LLMs?"
    )
)

Réponds à la question grâce au contexte si dessous. S'il n'y a pas de réponses possible à la question avec les informations fournies, 
réponds simplement "jsp".

Context:Les grands modèles de langage (LLM) sont les modèles les plus récents utilisés dans le domaine du NLP.
Leurs performances supérieures à celles des modèles plus petits les ont rendus incroyablement utiles pour les développeurs d'applications NLP.
Ces modèles sont accessibles via la bibliothèque `transformers` de Hugging Face, via OpenAI en utilisant la bibliothèque `openai`, 
et via Cohere en utilisant la bibliothèque `cohere`.

Question: Quelle bibliothèque et fournisseurs de modèles proposent des LLMs?

Answer: 


Et on peut ensuite le feed à notre LLM

In [13]:
print(openai.invoke(
    prompt_template.format(
        query="Quelle bibliothèque et fournisseurs de modèles proposent des LLMs?"
    )
))

La bibliothèque `transformers` de Hugging Face, OpenAI en utilisant la bibliothèque `openai` et Cohere en utilisant la bibliothèque `cohere`.


In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Raconte-moi un.e {adjective} blague sur {content}."
)

'Raconte-moi une sombre blague sur la police.'

In [15]:
print(openai.invoke(prompt_template.format(adjective="drole", content="la police")))



Pourquoi les policiers aiment-ils les escaliers ?

Parce qu'ils peuvent monter les marches sans être vus !


## `FewShotPromptTemplate`

In [18]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "Comment ca-va?",
        "answer": "Je ne suis pas à plaindre me je le fais quand-même."
    }, {
        "query": "Quel heure est-il?",
        "answer": "L'heure de vous acheter une montre."
    }, {
        "query": "À quoi sert un baromètre?",
        "answer": "À feur."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """Voici des extraits de conversations avec un assistant d'IA.
L'assistant est généralement sarcastique et plein d'esprit, produisant des réponses créatives et amusantes aux questions des utilisateurs. 
Voici quelques exemples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [20]:
query = "Quel est le sens de la vie?"

print(few_shot_prompt_template.format(query=query))

Voici des extraits de conversations avec un assistant d'IA.
L'assistant est généralement sarcastique et plein d'esprit, produisant des réponses créatives et amusantes aux questions des utilisateurs. 
Voici quelques exemples: 



User: Comment ca-va?
AI: Je ne suis pas à plaindre me je le fais quand-même.



User: Quel heure est-il?
AI: L'heure de vous acheter une montre.



User: Quel est le sens de la vie?
AI: 


In [24]:
query = "Quel est le sens de la vie?"
openai.invoke(few_shot_prompt_template.format(query=query))

"42, selon les Monty Python. Mais je pense que c'est plus compliqué que ça."

Autre exemple avec un assistant beaucoup plus concis

In [28]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "Comment ca-va?",
        "answer": "Ca-va."
    }, {
        "query": "Quel heure est-il?",
        "answer": "12:54"
    }, {
        "query": "À quoi sert un baromètre?",
        "answer": "À mesurer la pression."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """Voici des extraits de conversations avec un assistant d'IA.
L'assistant est sérieux et produits des réponses simples et courtes, d'un ou deux mots, aux questions des utilisateurs. 
Voici quelques exemples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [29]:
query = "Quel est la capitale de l'Angola?"
openai.invoke(few_shot_prompt_template.format(query=query))

'Luanda.'

# LLM et Chat Model

In [7]:
from langchain_openai import OpenAI

llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.9
    )
text = "Écris-moi un plan d'exposé sur la boxe" #prompt
print(llm.invoke(text))



I. Introduction 
    A. Présentation de la boxe
    B. Origines et évolution de la boxe 
    C. Objectif de l'exposé

II. Les règles de la boxe 
    A. Les catégories de poids 
    B. Les équipements nécessaires 
    C. Déroulement d'un combat de boxe 

III. Les différentes techniques de boxe 
    A. Le jab 
    B. Le crochet 
    C. L'uppercut 
    D. Les coups de pieds 

IV. Les bienfaits de la boxe 
    A. Pour la santé physique 
    B. Pour la santé mentale 
    C. La pédagogie et la discipline 

V. La boxe dans l'histoire 
    A. Les grands noms de la boxe 
    B. Les événements marquants 
    C. Les controverses autour de la boxe 

VI. La boxe professionnelle 
    A. Les différents titres 
    B. Le circuit professionnel 
    C. Les revenus des boxeurs 

VII. La boxe amateur 
    A. Les compétitions intern


## Chat Model et Chat message

In [31]:
from langchain_openai import ChatOpenAI

from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [36]:
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.8)

##### 1. Chat Messages

* HumanMessage: Un message envoyé du point de vue de l'utilisateur
* AIMessage: Message envoyé du point de vue du LLM avec lequel l'humain interagit.
* SystemMessage: Message définissant les objectifs que l'IA doit suivre
* ChatMessage: Message permettant de définir arbitrairement un rôle (rarement utilisé)

In [45]:
messages = [
    SystemMessage(content="Tu es un assistant IA efficace donnant des réponses courtes à l'utilisateurs."),
    HumanMessage(content="J'aime la montagne. Où devrais-je aller pour les vacances d'hiver?"),
    AIMessage(content="Les Alpes pour des vacances au ski semblent être une bonne idée?"),
    HumanMessage(content="J'aimerai connaitre le nombre d'espèces d'arachnides connues existant sur Terre.")
]
chat.invoke(messages)

AIMessage(content="Il existe environ 100 000 espèces d'arachnides connues à ce jour.")

# Output Parser

In [53]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.1,api_key=OPENAI_API_KEY)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="qestion pour faire rire.")
    punchline: str = Field(description="réponse pour faire la blague.")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Question mal formulé!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Réponds à la requête de l'utilisateurs.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Raconte-moi une blague."})
parser.invoke(output)

Joke(setup="Pourquoi les poissons n'aiment pas jouer au football ?", punchline="Parce qu'ils ont peur de prendre un carton jaune !")

pydantic.v1.main.PydanticOutputParserInput